In [1]:
import numpy as np
import torch
from einops import rearrange
from sklearn.decomposition import PCA
from tqdm.auto import tqdm

from diffusion_3d.chestct.autoencoder.vae.config import get_config
from diffusion_3d.chestct.autoencoder.vae.model import VAELightning
from diffusion_3d.datasets.ct_rate import CTRATEDataModule
from diffusion_3d.utils.visualize import plot_scans

In [2]:
config = get_config()
config

Munch({'data': Munch({'csvpath': '/raid3/arjun/ct_pretraining/csvs/sources.csv', 'datapath': '/raid3/arjun/ct_pretraining/scans/', 'checkpointspath': '/raid3/arjun/checkpoints/adaptive_autoencoder/', 'limited_dataset_size': None, 'allowed_spacings': ((0.4, 7), (-1, -1), (-1, -1)), 'allowed_shapes': ((64, -1), (256, -1), (256, -1)), 'train_augmentations': Munch({'_target_': 'monai.transforms.Compose', 'transforms': [Munch({'_target_': 'vision_architectures.transforms.croppad.CropForegroundWithCropTrackingd', 'keys': ['image'], 'source_key': 'image', 'allow_smaller': True}), Munch({'_target_': 'monai.transforms.ScaleIntensityRanged', 'keys': ['image'], 'a_min': -1000, 'a_max': 2000, 'b_min': -1.0, 'b_max': 1.0, 'clip': True}), Munch({'_target_': 'vision_architectures.transforms.croppad.RandSpatialCropSamplesWithCropTrackingd', 'keys': ['image'], 'roi_size': (51, 51, 51), 'max_roi_size': (76, 76, 76), 'random_size': True, 'num_samples': 5}), Munch({'_target_': 'vision_architectures.transf

In [4]:
checkpoint_path = r"/raid3/arjun/checkpoints/adaptive_autoencoder/v42__2025_03_24/version_0/checkpoints/last.ckpt"

model = VAELightning.load_from_checkpoint(
    checkpoint_path,
    map_location="cpu",
    model_config=config.model,
).autoencoder
model.eval()


VAE(
  (encoder): SwinV23DModel(
    (patchify): PatchEmbeddings3D(
      (patch_embeddings): Conv3d(1, 24, kernel_size=(2, 2, 2), stride=(2, 2, 2))
      (normalization): LayerNorm((24,), eps=1e-05, elementwise_affine=True)
    )
    (absolute_position_embeddings): AbsolutePositionEmbeddings3D()
    (encoder): SwinV23DEncoder(
      (stages): ModuleList(
        (0): SwinV23DStage(
          (blocks): ModuleList(
            (0-1): 2 x SwinV23DBlock(
              (w_layer): SwinV23DLayer(
                (transformer): Attention3DWithMLP(
                  (attn): Attention3D(
                    (W_q): Linear(in_features=24, out_features=24, bias=True)
                    (W_k): Linear(in_features=24, out_features=24, bias=True)
                    (W_v): Linear(in_features=24, out_features=24, bias=True)
                    (proj): Linear(in_features=24, out_features=24, bias=True)
                    (proj_drop): Dropout(p=0.1, inplace=False)
                    (logit_scale): Sw

In [5]:
datamodule = CTRATEDataModule(config.data)
dataloader = datamodule.val_dataloader()
len(dataloader)

valid:   0%|          | 0/500 [00:00<?, ?it/s]

No. of valid datapoints: 500


500

In [6]:
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        if i >= 1:
            break

        x = batch["image"][:4]
        crop_offsets = batch['crop_offset'][:4]

        reconstructed = model(x, crop_offsets, "valid")["reconstructed"].cpu()
        print(f"Batch idx: {i}")
        print(reconstructed.min(), reconstructed.max())
        print(x.shape, reconstructed.shape)
        print((x.min(), x.max()), (reconstructed.min(), reconstructed.max()))

        x[:, :, :, 0, 0] = -1.0
        x[:, :, :, -1, -1] = 1.0
        reconstructed[:, :, :, 0, 0] = -1.0
        reconstructed[:, :, :, -1, -1] = 1.0

        for i in range(x.shape[0]):
            plot_scans([x[i][0], reconstructed[i][0]], ["Original", "Reconstructed"])

torch.cuda.empty_cache()

Batch idx: 0
metatensor(-0.7735) metatensor(0.3016)
torch.Size([4, 1, 64, 64, 64]) torch.Size([4, 1, 64, 64, 64])
(metatensor(-1.), metatensor(0.7240)) (metatensor(-0.7735), metatensor(0.3016))


interactive(children=(IntSlider(value=0, description='z', max=63), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=63), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=63), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=63), Output()), _dom_classes=('widget-interact',…

# Whole CT

In [ ]:
raise Exception("Don't want to automatically run beyond this point")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 raise Exception("Don't want to automatically run beyond this point")                         │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
Exception: Don't want to automatically run beyond this point

In [ ]:
dataloader = datamodule.test_dataloader()
len(dataloader)

test:   0%|          | 0/500 [00:00<?, ?it/s]

No. of test datapoints: 500


500

In [ ]:
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        if i < 1:
            continue

        if i >= 2:
            break

        x = batch["image"]
        print(x.shape)
        x = x[:, :, :, 128:384, 128:384]
        print(x.shape)

        model.cuda()
        x = x.float()

        reconstructed = model(x.cuda(), "valid")["reconstructed"].cpu()
        print(f"Batch idx: {i}")
        print(reconstructed.min(), reconstructed.max())
        print(x.shape, reconstructed.shape)
        print((x.min(), x.max()), (reconstructed.min(), reconstructed.max()))

        reconstructed[:, :, :, 0, 0] = -1.0
        reconstructed[:, :, :, -1, -1] = 1.0

        for i in range(x.shape[0]):
            plot_scans([x[i][0], reconstructed[i][0]], ["Original", "Reconstructed"])

torch.cuda.empty_cache()

torch.Size([1, 1, 512, 512, 512])
torch.Size([1, 1, 512, 256, 256])
Batch idx: 1
metatensor(-0.9945) metatensor(0.5118)
torch.Size([1, 1, 512, 256, 256]) torch.Size([1, 1, 512, 256, 256])
(metatensor(-1.), metatensor(1.)) (metatensor(-0.9945), metatensor(0.5118))


interactive(children=(IntSlider(value=0, description='z', max=511), Output()), _dom_classes=('widget-interact'…

# PCA

In [ ]:
raise Exception("Don't want to automatically run beyond this point")

In [ ]:
import gc
from collections import defaultdict

device = torch.device("cuda:0")

latent_vectors = []
stage_output_vectors = defaultdict(list)

model.to(device)

with torch.no_grad():
    for batch in tqdm(dataloader):
        x = batch["image"].to(device)
        mu, _, stage_outputs = model.encode(x, return_stage_outputs=True)
        x.cpu()
        del x
        latent_vectors.append(mu.cpu().numpy())
        for i in range(len(stage_outputs)):
            stage_output_vectors[i].append(stage_outputs[i].cpu().numpy())

gc.collect()
torch.cuda.empty_cache()

latent_vectors = np.concatenate(latent_vectors, axis=0)

  0%|          | 0/500 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11 with torch.no_grad():                                                                       │
│   12 │   for batch in tqdm(dataloader):                                                          │
│   13 │   │   x = batch["image"].to(device)                                                       │
│ ❱ 14 │   │   mu, _, stage_outputs = model.encode(x, return_stage_outputs=True)                   │
│   15 │   │   x.cpu()                                                                             │
│   16 │   │   del x                                                                               │
│   17 │   │   latent_vectors.append(mu.cpu().numpy())                                             │
│                                                                                                  │
│ /home/ubuntu/projects/diffusion_3d/src/diffusion_3d/chestct/autoencoder/vae/nn.py:212 in encode  │
│                                                                                                  │
│   209 │   │   #     sliding_window = 2**14                                                       │
│   210 │   │   #     sliding_stride = sliding_window // 2                                         │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   adapted = self.adapt(                                                              │
│   213 │   │   │   list(reversed(stage_outputs)),  # Show low frequency features first, then sh   │
│   214 │   │   │   sliding_window=sliding_window,                                                 │
│   215 │   │   │   sliding_stride=sliding_stride,                                                 │
│                                                                                                  │
│ /home/ubuntu/miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py:1931 in          │
│ __getattr__                                                                                      │
│                                                                                                  │
│   1928 │   │   │   modules = self.__dict__["_modules"]                                           │
│   1929 │   │   │   if name in modules:                                                           │
│   1930 │   │   │   │   return modules[name]                                                      │
│ ❱ 1931 │   │   raise AttributeError(                                                             │
│   1932 │   │   │   f"'{type(self).__name__}' object has no attribute '{name}'"                   │
│   1933 │   │   )                                                                                 │
│   1934                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'AdaptiveVAE' object has no attribute 'adapt'

In [ ]:
pattern = "b c z y x -> (b z y x) c"
# pattern = "b c z y x -> b (c z y x)"

latent_vectors_rearranged = rearrange(latent_vectors, pattern)
for i in range(len(stage_output_vectors)):
    stage_output_vectors[i] = np.concatenate(stage_output_vectors[i], axis=0)
    stage_output_vectors[i] = rearrange(stage_output_vectors[i], pattern)

for name, fit_vectors in [(f"Stage {i+1}", stage_output_vectors[i]) for i in range(len(stage_output_vectors))] + [
    ("Sampled", latent_vectors_rearranged)
]:
    try:
        # Compute PCA
        pca = PCA()
        pca.fit(fit_vectors)

        # Calculate effective dimensionality
        explained_variance_ratio = pca.explained_variance_ratio_
        cumulative_variance = np.cumsum(explained_variance_ratio)
        effective_dim = np.argmax(cumulative_variance > 0.95) + 1

        print(
            f"Effective dim {name}: {effective_dim} / {fit_vectors.shape[1]}\t({int(effective_dim * 100 / fit_vectors.shape[1])}%)"
        )
    except Exception as e:
        print(f"Failed to compute PCA for {name}: {e}")

# Visualize patches

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
patch_size = 4

x = batch["scan"]

plt.figure(figsize=(10, 10))
plt.imshow(x[0][0][128], cmap="gray")
plt.grid(color="r", linestyle="-", linewidth=0.3)
plt.gca().set_xticks(np.arange(0, x.shape[-1], patch_size))
plt.gca().set_yticks(np.arange(0, x.shape[-1], patch_size))
plt.show()

# Check latent space

In [ ]:
raise Exception("Don't want to automatically run beyond this point")

In [ ]:
dataloader_iter = iter(dataloader)
batch1 = next(dataloader_iter)
batch2 = next(dataloader_iter)
x1, x2 = batch1["scan"], batch2["scan"]
(batch1["uid"], batch2["uid"]), (x1.shape, x2.shape)

In [ ]:
with torch.no_grad():
    output1 = model.process_step(x1, [], "valid", 0)
    output2 = model.process_step(x2, [], "valid", 0)

reconstructed1 = output1["reconstructed"]
reconstructed2 = output2["reconstructed"]

adapted_encoded1 = output1["adapted_encoded"]
adapted_encoded2 = output2["adapted_encoded"]

(reconstructed1.shape, reconstructed2.shape), (adapted_encoded1.shape, adapted_encoded2.shape)

In [ ]:
adapted_encoded_inter = (adapted_encoded1 + adapted_encoded2) / 2
adapted_encoded_inter.shape

In [ ]:
from torch.nn import functional as F

with torch.no_grad():
    decoded_inter = model.decode(adapted_encoded_inter)
    reconstructed_inter = F.interpolate(decoded_inter, x1.shape[2:], mode="trilinear")
reconstructed_inter.shape

In [ ]:
plot_scans(
    [x1[0][0], x2[0][0], reconstructed1[0][0], reconstructed2[0][0], reconstructed_inter[0][0]],
    ["Scan1", "Scan2", "Reconstructed1", "Reconstructed2", "Reconstructed interpolated latent"],
    cols=2,
)

In [ ]:
plot_scans((reconstructed1 - reconstructed2).abs()[0][0])